In [2]:
import paramiko

class SftpCon(object):
    def __init__(self, hostname, username, password=None):
        self.hostname = hostname
        self.username = username
        self.password = password
        self.ssh_client = paramiko.SSHClient()
        self.ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.ssh_client.connect(hostname, username=username, password=password)
        self.sftp_client = self.ssh_client.open_sftp()

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.close()

    def close(self):
        self.sftp_client.close()
        self.ssh_client.close()

    def get(self, remote_path, local_path):
        self.sftp_client.get(remote_path, local_path)

    def put(self, local_path, remote_path):
        self.sftp_client.put(local_path, remote_path)

    def ls(self):
        path = "."
        return self.sftp_client.listdir(path)

In [4]:
from pathlib import Path
import os
import time

hostname = '192.168.0.166'
username = 'pi'
remote_path = "/home/pi/remote_dataframe.csv"
local_path = "local_dataframe.csv"

with SftpCon(hostname, username) as sftp:
    # Show timestamp on local file
    ts = os.path.getmtime("local_dataframe.csv")
    print(f"local_dataframe.csv: {time.ctime(ts)}")

    sftp.get(remote_path, local_path)

    # Show timestamp on local file
    ts = os.path.getmtime("local_dataframe.csv")
    print(f"local_dataframe.csv: {time.ctime(ts)}")

    # Show timestamp on remote file
    ts = sftp.sftp_client.stat(remote_path).st_mtime
    print(f"remote_dataframe.csv: {time.ctime(ts)}")

    sftp.put(local_path, remote_path)

    # Show timestamp on remote file
    ts = sftp.sftp_client.stat(remote_path).st_mtime
    print(f"remote_dataframe.csv: {time.ctime(ts)}")


local_dataframe.csv: Mon Mar 11 21:32:49 2024
local_dataframe.csv: Mon Mar 11 21:35:26 2024
remote_dataframe.csv: Mon Mar 11 21:32:51 2024
remote_dataframe.csv: Mon Mar 11 21:35:29 2024
